In [1]:
using GpABC, DifferentialEquations, Distributions, Distances, Plots

In [2]:
#
# Experimental data - from ABCSysBio example at
# https://github.com/jamesscottbrown/abc-sysbio/blob/master/examples/SBML/Example1/SIR.ipynb
#
times = [0.0, 0.6, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]
values = [[ 20.     ,  10.     ,   0.     ],
       [  0.12313,  13.16813,   9.42344],
       [  0.12102,   7.17251,  11.18957],
       [  0.09898,   2.36466,  10.0365 ],
       [  0.37887,   0.92019,   6.87117],
       [  1.00661,   0.61958,   4.44955],
       [  1.20135,   0.17449,   3.01271],
       [  1.46433,   0.28039,   1.76431],
       [  1.37789,   0.0985 ,   1.28868],
       [  1.57073,   0.03343,   0.81813],
       [  1.4647 ,   0.28544,   0.52111],
       [  1.24719,   0.10138,   0.22746],
       [  1.56065,   0.21671,   0.19627]]

values = hcat(values...)

#
# Priors and initial conditions - these are model-specfic as each model can 
# have different numbers of parameters/species
#
priors1 = [Uniform(0.0, 5.0) for i in 1:4]
ic1 = [20.0, 10.0, 0.0]

priors2 = vcat([Uniform(0.0, 5.0) for i in 1:4], Uniform(0.0, 10.0))
ic2 = [20.0, 0.0, 10.0, 0.0]

priors3 = vcat([Uniform(0.0, 5.0) for i in 1:4], Uniform(0.0, 10.0))
ic3 = [20.0, 10.0, 0.0]

#modelprior = DiscreteUniform(1, 3)
modelprior = DiscreteUniform(1, 2)

# p = (alpha, gamma, d, v)
# x = (S, I, R)
function model1(dx, x, p, t)
    dx[1] = p[1] - p[2]*x[1]*x[2] - p[3]*x[1] # dS/dt = alpha - gamma*S*I - d*S
    dx[2] = p[3]*x[1]*x[2] - p[4]*x[2] - p[3]*x[2] # dI/dt = gamma*S*I - v*I - d*I
    dx[3] = p[4]*x[2] - p[3]*x[3] # dR/dt = v*I - d*R
end

# p = (alpha, gamma, d, v, delta)
# x = (S, L, I, R)
function model2(dx, x, p, t)
    dx[1] = p[1] - p[2]*x[1]*x[3] - p[3]*x[1] # dS/dt = alpha - gamma*S*I - d*S
    dx[2] = p[2]*x[1]*x[3] - p[5]*x[2] - p[3]*x[2] # dL/dt = gamma*S*I - delta*L - d*L
    dx[3] = p[5]*x[2] - p[4]*x[3] - p[3]*x[3] # dI/dt = delta*L - v*I - d*I
    dx[4] = p[4]*x[3] - p[3]*x[4] # dR/dt = v*I - d*R
end

# p = (alpha, gamma, d, v, e)
# x = (S, I, R)
function model3(dx, x, p, t)
    dx[1] = p[1] - p[2]*x[1]*x[2] - p[3]*x[1] + p[5]*x[3] # dS/dt = alpha - gamma*S*I - d*S + e*R
    dx[2] = p[3]*x[1]*x[2] - p[4]*x[2] - p[3]*x[2] # dI/dt = gamma*S*I - v*I - d*I
    dx[3] = p[4]*x[2] - (p[3]+p[5])*x[3] # dR/dt = v*I - (d+e)*R
end

#ics = [ic1, ic2, ic3]
ics = [ic1, ic2]

# Define simulator functions

simulator1(params) = Array{Float64,2}(
    solve(ODEProblem(model1, ics[1], (times[1], times[end]), params), saveat=times, force_dtmin=true))

# Model2 contains the species L, which is not measured - we remove it from the returned ODE solution
# so that it can be compared to the reference data "values", which only contains S, I and R
simulator2(params) = Array{Float64,2}(
    solve(ODEProblem(model2, ics[2], (times[1], times[end]), params), saveat=times, force_dtmin=true))[[1,3,4],:]

simulator3(params) = Array{Float64,2}(
    solve(ODEProblem(model3, ics[3], (times[1], times[end]), params), saveat=times, force_dtmin=true))

simulator3 (generic function with 1 method)

In [3]:
if false
    t_schedule = [20, 15, 10, 5, 3, 2.5, 2, 1.7, 1.5]
    #t_schedule = [20, 15, 10, 5]

    input = GpABC.SimulatedModelSelectionInput(2, 200, t_schedule, modelprior,
        [priors1, priors2], "keep_all", euclidean,
        [simulator1, simulator2], 1e4)

    ms_res = GpABC.model_selection(input, values);
end

In [11]:
threshold_schedule = [20, 15, 10, 5, 3, 2.5, 2, 1.7, 1.5]

# Train the emulators
n_design_points = 200
summary_statistic = "keep_all"
distance_metric = euclidean
rt = RepetitiveTraining()
gpkernel = SquaredExponentialArdKernel()

#
# A set of functions that return a trained emulator with a prior sampling function as an argument
#
emulator_trainers = [f(prior_sampler) = GpABC.abc_train_emulator(prior_sampler,
        n_design_points,
        GpABC.build_summary_statistic(summary_statistic)(values),
        sim,
        GpABC.build_summary_statistic(summary_statistic),
        distance_metric)
    for sim in [simulator1, simulator2]]

emulator_settings = [AbcEmulationSettings(n_design_points,
        trainer,
        (x, em) -> gp_regression(x, em)) for trainer in emulator_trainers]

input = EmulatedModelSelectionInput(2, 200, threshold_schedule, modelprior, [priors1, priors2],
    emulator_settings, 100, 1e3)

ms_res = model_selection(input, values)

Emulated model selection
Population 1 - ABC Rejection ϵ = 20.0
Trained emulators
Completed 13 iterations, accepting 200 particles in total
Number of accepted parameters: Model 1: 109	Model 2: 91
Population 2 - ABCSMC ϵ = 15.0
Completed 63 iterations, accepting 200 particles
Number of accepted parameters: Model 1: 68	Model 2: 132
Population 3 - ABCSMC ϵ = 10.0
Completed 509 iterations, accepting 200 particles
Number of accepted parameters: Model 1: 130	Model 2: 70
Population 4 - ABCSMC ϵ = 5.0
Completed 1000 iterations, accepting 89 particles
Number of accepted parameters: Model 1: 45	Model 2: 44
Population 5 - ABCSMC ϵ = 3.0


Completed 1000 iterations, accepting 2 particles
Number of accepted parameters: Model 1: 0	Model 2: 2
Population 6 - ABCSMC ϵ = 2.5


Completed 1000 iterations, accepting 0 particles
Number of accepted parameters: Model 1: 0	Model 2: 0


GpABC.ModelSelectionOutput(2, AbstractArray{Int64,1}[[109, 91], [68, 132], [130, 70], [45, 44], [0, 2], [0, 0]], [20.0, 15.0, 10.0, 5.0, 3.0, 2.5], GpABC.ABCSMCOutput[GpABC.EmulatedABCSMCOutput(4, [109, 68, 130, 45, 0, 0], [316, 0, 0, 0, 0, 0], [20.0, 15.0, 10.0, 5.0, 3.0, 2.5], AbstractArray{Float64,2}[[2.51932 2.18741 0.653133 0.524235; 2.29531 4.29587 2.10961 2.22487; … ; 4.98631 2.81775 1.90816 1.22571; 3.58046 4.00558 2.11353 2.24198], [4.56909 3.34787 0.272535 0.898501; 0.180029 1.05551 2.08302 3.01319; … ; 4.44748 1.14291 2.12755 2.52634; 0.809684 0.928728 1.20651 2.8923], [0.10498 0.750214 1.80512 2.79711; 1.40829 0.43629 0.360746 0.00699345; … ; 3.89482 0.832674 1.97042 0.608193; 2.55265 0.616259 1.02064 0.00521675], [3.91806 0.956576 1.6965 0.442793; 3.35418 0.598678 1.01315 0.109975; … ; 0.0752068 0.750007 1.12923 1.49538; 3.64069 0.79002 0.645045 0.294499], Array{Float64}(0,4), Array{Float64}(0,4)], AbstractArray{Float64,1}[[17.2115, 19.4157, 19.9811, 18.06, 17.7523, 19.840

ERROR (unhandled task failure): InterruptException:
Stacktrace:
 [1] process_events at ./libuv.jl:82 [inlined]
 [2] wait() at ./event.jl:216
 [3] wait(::Condition) at ./event.jl:27
 [4] stream_wait(::Timer, ::Condition, ::Vararg{Condition,N} where N) at ./stream.jl:42
 [5] wait(::Timer) at ./event.jl:357
 [6] (::Base.##300#301{IJulia.#send_stderr,Timer})() at ./event.jl:430


In [5]:
ms_res.smc_outputs[1]

GpABC.EmulatedABCSMCOutput(4, [0, 0, 0, 0, 0, 0, 0], [669, 0, 0, 0, 0, 0, 0], [20.0, 15.0, 10.0, 5.0, 3.0, 2.5, 2.0], AbstractArray{Float64,2}[Array{Float64}(0,4), Array{Float64}(0,4), Array{Float64}(0,4), Array{Float64}(0,4), Array{Float64}(0,4), Array{Float64}(0,4), Array{Float64}(0,4)], AbstractArray{Float64,1}[Float64[], Float64[], Float64[], Float64[], Float64[], Float64[], Float64[]], StatsBase.Weights[Float64[], Float64[], Float64[], Float64[], Float64[], Float64[], Float64[]], Any[GpABC.GPModel(GpABC.SquaredExponentialArdKernel(GpABC.RbfCovarianceCache([5.89913, 9.99999, -1.17922, 4.31046, 9.31052], [0.0 181.237 … 3.49861 81.2068; 181.237 0.0 … 134.382 19.811; … ; 3.49861 134.382 … 7.10543e-15 50.9994; 81.2068 19.811 … 50.9994 0.0], [1.3302e5 5.87251e-35 … 23131.4 3.09094e-13; 5.87251e-35 1.3302e5 … 8.77475e-25 6.63775; … ; 23131.4 8.77475e-25 … 1.3302e5 1.12083e-6; 3.09094e-13 6.63775 … 1.12083e-6 1.3302e5])), GpABC.HPOptimisationCache([5.89913, 9.99999, -1.17922, 4.31046, 9.3

In [6]:
function prior_sampler(n_dp, priors)
    println("n_dp = $n_dp")
    out = hcat([rand(p, n_dp) for p in priors]...)
    println(size(out))
    return out
end

f1(x) = emulator_trainers[1](x -> prior_sampler(x, priors1))
f2(x) = emulator_trainers[2](x -> prior_sampler(x, priors2))
f3(x) = emulator_trainers[3](x -> prior_sampler(x, priors3))

# f1(2)
# f2(10)
# f3(10)

f(x) = prior_sampler(x, priors1)
f1(10)
f(10)

n_dp = 200
(200, 4)
n_dp = 10
(10, 4)


10×4 Array{Float64,2}:
 2.32416  3.6835    4.03283     0.709933
 1.09039  2.55734   0.00603134  3.21952 
 1.88055  0.536153  2.81769     1.99295 
 4.02369  3.94946   1.52606     2.83199 
 1.64954  4.90941   3.17308     3.84221 
 1.84796  3.24773   2.07166     4.52244 
 1.36753  1.29882   0.858871    4.79825 
 1.24243  4.49825   2.63612     0.787756
 3.57589  4.13275   4.69196     0.134196
 1.34541  4.73936   4.40642     4.16915 

In [7]:
using StatsBase

w1 = Weights([0.1,0.3])
vcat([0.1], w1)

3-element Array{Float64,1}:
 0.1
 0.1
 0.3

In [8]:
n_design_points = 200
X = hcat([rand(p, n_design_points) for p in priors3]...)
sum_stat = GpABC.build_summary_statistic(summary_statistic)
sum_ref = sum_stat(values)
y = [distance_metric(sum_stat(simulator3(X[i,:])), sum_ref) for i = 1:n_design_points]
gpem = GPModel(training_x = X, training_y = y)
gp_train(gpem)
fhat, V = gp_regression(X, gpem, full_covariance_matrix=true)

LoadError: [91mBoundsError: attempt to access 2-element Array{Array{Float64,1},1} at index [3][39m

In [9]:
println(y)
println()
f_draw = rand(MvNormal(fhat, Array(Hermitian(V))))
println(f_draw)

LoadError: [91mUndefVarError: y not defined[39m

In [17]:
threshold_schedule = [20, 15, 10, 5, 3, 2.5, 2, 1.7, 1.5]

# Train the emulators
n_design_points = 200
summary_statistic = "keep_all"
distance_metric = euclidean

function model_selection(
    reference_data::AbstractArray{Float64,2},
    n_particles::Integer,
    threshold_schedule::AbstractArray{Float64,1},
    parameter_priors::AbstractArray{AD,1},
    summary_statistic::Union{String,AbstractArray{String,1},Function},
    simulator_functions::AbstractArray{Function,1};
    model_prior::DiscreteUnivariateDistribution=Distributions.DiscreteUniform(1,length(parameter_priors)),
    distance_function::Function=Distances.euclidean,
    max_iter=1e4
    ) where {
    D<:ContinuousUnivariateDistribution,
    AD<:AbstractArray{D,1}
    }

    input = GpABC.SimulatedModelSelectionInput(length(parameter_priors), n_particles, threshold_schedule, model_prior,
        parameter_priors, summary_statistic, distance_function,
        simulator_functions, max_iter)

    return model_selection(input, reference_data)
end

ms_res  = model_selection3(values, 200, threshold_schedule, [priors1, priors2],
    "keep_all", [simulator1, simulator2])

Simulated model selection
Population 1
Completed 689 iterations, accepting 200 particles in total
Number of accepted parameters: Model 1: 97	Model 2: 103
Population 2
Completed 1315 iterations, accepting 200 particles
Number of accepted parameters: Model 1: 52	Model 2: 148
Population 3
Completed 2990 iterations, accepting 200 particles
Number of accepted parameters: Model 1: 30	Model 2: 170
Population 4
Completed 10000 iterations, accepting 187 particles
Number of accepted parameters: Model 1: 51	Model 2: 136
Population 5


Completed 7302 iterations, accepting 200 particles
Number of accepted parameters: Model 1: 107	Model 2: 93
Population 6
Completed 1975 iterations, accepting 200 particles
Number of accepted parameters: Model 1: 108	Model 2: 92
Population 7
Completed 2560 iterations, accepting 200 particles
Number of accepted parameters: Model 1: 124	Model 2: 76
Population 8
Completed 2634 iterations, accepting 200 particles
Number of accepted parameters: Model 1: 129	Model 2: 71
Population 9
Completed 2713 iterations, accepting 200 particles
Number of accepted parameters: Model 1: 123	Model 2: 77


GpABC.ModelSelectionOutput(2, AbstractArray{Int64,1}[[97, 103], [52, 148], [30, 170], [51, 136], [107, 93], [108, 92], [124, 76], [129, 71], [123, 77]], [20.0, 15.0, 10.0, 5.0, 3.0, 2.5, 2.0, 1.7, 1.5], GpABC.ABCSMCOutput[GpABC.SimulatedABCSMCOutput(4, [97, 52, 30, 51, 107, 108, 124, 129, 123], [339, 700, 1513, 4934, 3650, 948, 1247, 1289, 1356], [20.0, 15.0, 10.0, 5.0, 3.0, 2.5, 2.0, 1.7, 1.5], AbstractArray{Float64,2}[[2.08642 1.88071 1.9823 0.218942; 4.20892 1.61144 1.66895 2.4996; … ; 3.22482 3.57472 1.05823 2.23355; 3.29277 1.05655 0.876423 2.01265], [1.23971 1.69343 0.569956 2.06394; 4.11027 0.875915 0.738888 1.53348; … ; 3.69149 0.822397 1.96732 1.17392; 4.09696 0.935283 0.534805 0.474012], [0.0449582 0.437854 0.805787 1.03709; 4.32242 0.911089 0.906442 1.16847; … ; 2.92539 0.642313 1.05107 0.925451; 2.24981 0.457438 1.11067 1.08566], [1.19538 0.544699 0.411286 1.21346; 1.31124 0.508125 0.517961 0.796634; … ; 2.53974 0.580897 0.676828 0.983284; 1.06163 0.465576 0.397157 1.05019]

In [20]:
typeof(10000)

Int64